# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker','Stock Price','Market Capitilization','Number of Shares to Buy']
final_df = pd.DataFrame(columns=my_columns)
final_df

,Ticker,Stock Price,Market Capitilization,Number of Shares to Buy


In [7]:
series = pd.Series([symbol,price,market_cap,'N/A'],index=my_columns)
final_df.append(series,ignore_index=True)

,Ticker,Stock Price,Market Capitilization,Number of Shares to Buy
0,AAPL,186.42,3041419199387,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
# final_df = pd.DataFrame(columns=my_columns)
# for symbol in stocks['Ticker']:
#     api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
#     data = requests.get(api_url).json()
#     price = data['latestPrice']
#     market_cap = data['marketCap']
#     series = pd.Series([symbol,price,market_cap,'N/A'],index=my_columns)
#     final_df = final_df.append(series,ignore_index=True)

In [9]:
# final_df.head()

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [11]:
final_df = pd.DataFrame(columns=my_columns)
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for series in symbol_groups:
    symbol_strings.append(','.join(series))

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        price = data[symbol]['quote']['latestPrice']
        market_cap = data[symbol]['quote']['marketCap']
        series = pd.Series([symbol,price,market_cap,'N/A'],index=my_columns)
        final_df = final_df.append(series,ignore_index=True)
        
final_df.head()

,Ticker,Stock Price,Market Capitilization,Number of Shares to Buy
0,A,167.08,50172066326,N/A
1,AAL,18.66,12177308250,N/A
2,AAP,251.57,15035978935,N/A
3,AAPL,181.22,2957536029975,N/A
4,ABBV,138.60,249283307969,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [12]:
investment_size = float(input('Enter the value of your investment'))

Enter the value of your investment100000000


In [13]:
position_size = investment_size/final_df.shape[0]

In [14]:
final_df['Number of Shares to Buy'] = position_size/final_df['Stock Price']
final_df.head()

,Ticker,Stock Price,Market Capitilization,Number of Shares to Buy
0,A,167.08,50172066326,1185.179567
1,AAL,18.66,12177308250,10611.993675
2,AAP,251.57,15035978935,787.135994
3,AAPL,181.22,2957536029975,1092.703907
4,ABBV,138.60,249283307969,1428.714300


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [15]:
writer = pd.ExcelWriter('recommended_trades.xlsx',engine='xlsxwriter')
final_df.to_excel(writer,sheet_name='Recommended Trades',index=False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [16]:
background_color = '#0a0a23'
font_color = 'ffffff'

string_format = writer.book.add_format({'font_color':font_color,'bg_color':background_color,'border':1})
dollar_format = writer.book.add_format({'num_format':'$0.00','font_color':font_color,'bg_color':background_color,'border':1})
float_format = writer.book.add_format({'num_format':'0.000000','font_color':font_color,'bg_color':background_color,'border':1})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [17]:
# writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
# writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)

# writer.sheets['Recomended Trades'].write('A1','Ticker',string_format)
# writer.sheets['Recomended Trades'].write('B1','Stock Price',string_format)
# writer.sheets['Recomended Trades'].write('C1','Market Capitilization',string_format)
# writer.sheets['Recomended Trades'].write('D1','Number of Shares to Buy',string_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [18]:
column_formats = {'A':['Ticker',string_format],'B':['Stock Price',dollar_format],
                  'C':['Market Capitilization',dollar_format],'D':['Number of Shares to Buy',float_format]}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],string_format)
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])    

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()